In [ ]:
import os
import sys
import torch
import time
import shutil
from torchvision.transforms import ToPILImage
from os.path import join

base_directory = "../"
sys.path.insert(0, base_directory)

output_base_dir = join(base_directory, "./output/sd2-notebook/")
output_directory = join(output_base_dir, "unet/")

try:
    shutil.rmtree(output_directory)
except Exception as e:
    print(e, "\n", "Creating the path...")
    os.makedirs(output_directory, exist_ok=True)
else:
    os.makedirs(output_directory, exist_ok=True)

from stable_diffusion.stable_diffusion import StableDiffusion
from stable_diffusion.utils_backend import *
from stable_diffusion.utils_image import *
from stable_diffusion.utils_model import *
from stable_diffusion.utils_logger import *
from stable_diffusion.constants import IODirectoryTree


to_pil = lambda image: ToPILImage()(torch.clamp((image + 1.0) / 2.0, min=0.0, max=1.0))

In [ ]:
device = get_device()

In [ ]:
pt = IODirectoryTree(base_directory=base_directory)

In [ ]:
# initialize an empty stable diffusion class
stable_diffusion = StableDiffusion(device=device)
get_memory_status()

In [ ]:
# initialize an empty latent diffusion model; it returns self.model
stable_diffusion.quick_initialize()
get_memory_status()

In [ ]:
# to use the unet we first need conditioning context. we can use the clip embedder to get it.
stable_diffusion.model.load_clip_embedder(**pt.embedder)
get_memory_status()

In [ ]:
# load the embedder submodels, tokenizer and transformer
stable_diffusion.model.clip_embedder.load_submodels(**pt.embedder_submodels)
get_memory_status()

In [ ]:
stable_diffusion.model.clip_embedder

In [ ]:
# get the embedding for a prompt
prompt_embedding = stable_diffusion.model.clip_embedder(
    ["A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta"]
)

In [ ]:
get_memory_status()
prompt_embedding.shape

In [ ]:
# we don't need the embedder anymore, so we can unload it
stable_diffusion.model.unload_clip_embedder()
get_memory_status()

In [ ]:
# let's save the prompt embedding
torch.save(prompt_embedding, join(output_base_dir, "prompt_embedding.pt"))

In [ ]:
# the latent diffusion class has a method to load the unet, since it is a submodel of it. it returns the unet model, wrapped in a DiffusionWrapper class.
# it is accessible as self.model.model or through the alias self.model.unet
stable_diffusion.model.load_unet(**pt.unet)
get_memory_status()
stable_diffusion.model.unet

In [ ]:
# load an encoded image and get its shape
encoded_img = torch.load(join(output_base_dir, "encoded_img_tensor.pt")).to(device)
encoded_img.shape

In [ ]:
# sample a latent representation of same shape as the encoded image
sample = torch.randn_like(encoded_img)
get_memory_status()

In [ ]:
# define a timestep for this sample
time_step = torch.tensor([0.0]).to(device)
time_step.shape

In [ ]:
# predict noise with the unet
unet_output = stable_diffusion.model.unet(sample, time_step, prompt_embedding)

In [ ]:
get_memory_status()
unet_output.shape

In [ ]:
grid = torchvision.utils.make_grid(
    unet_output.permute(1, 0, 2, 3),
    nrow=2,
    normalize=False,
    range=(-1, 1),
    scale_each=True,
    pad_value=0,
)
dim_grid_image = to_pil(grid)
dim_grid_image.save(join(output_directory, f"unet_output.png"))
dim_grid_image

In [ ]:
stable_diffusion.model.unload_submodels()
torch.cuda.empty_cache()
get_memory_status()